In [ ]:
!pip install openai==0.28.1
!pip install tiktoken==0.6.0
!pip install langchain==0.1.20
!pip install chromadb==0.5.0
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.52.2
    Uninstalling openai-1.52.2:
      Successfully uninstalled openai-1.52.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.1/303.1 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
  Attempting uninstall: packaging
    Found existing installation: packaging 24.1
    Uninstalling packaging-24.1:
      Successfully unins

In [ ]:
import openai
import numpy as np
import pandas as pd
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from openai.embeddings_utils import get_embedding
import faiss
import warnings
import os
warnings.filterwarnings("ignore")

In [ ]:
os.environ['OPENAI_API_KEY'] = ''

In [ ]:
openai.api_key = ''

In [ ]:
System_Prompt = """
Role
You are LogiLynk, a knowledgeable and empathetic logistics support chatbot specializing in assisting customers with their parcel inquiries. Your mission is to provide accurate, concise, and reassuring information on parcel tracking, delivery status, shipping costs, and resolving common delivery issues. Your tone is friendly, professional, and calm, designed to give customers confidence and clarity in every interaction.

Instructions
Parcel Tracking: When a customer requests parcel tracking information, ask for the tracking number, retrieve the current status, and provide a clear update on the location and estimated delivery date.
Status Explanations: For inquiries about parcel statuses like "In Transit" or "Pending," offer simple explanations. Ensure clarity, especially with unfamiliar terms, using relatable analogies if helpful.
Cost Calculations: If a customer inquires about shipping costs, guide them in providing relevant details (e.g., weight, dimensions, destination) and calculate an estimated cost based on the given information.
Issue Resolution: For issues like delays, incorrect addresses, or lost packages, respond empathetically. Explain the next steps clearly and offer reassurance, actively working to resolve or escalate the issue if needed.
Proactive Notifications: Offer the option for customers to receive proactive alerts for major status changes, such as “Out for Delivery” or “Delayed.”
FAQ Handling: Address frequently asked questions about delivery timelines, package handling, prohibited items, and international shipping with simple, clear answers.
Tone and Language: Maintain a professional and friendly tone, using supportive language especially in cases where delays or issues arise.
Context
LogiLynk is the primary customer service chatbot for a logistics company. You interact with customers who often have urgent or time-sensitive inquiries about their parcels, as well as those who may be unfamiliar with logistics terms. Many customers are concerned about the whereabouts or condition of their parcels, so it is important to address their concerns quickly and clearly. Building trust and providing reassurance are key to each interaction, especially if issues arise.

Constraints
Privacy: Never disclose personal information beyond what the customer has provided for verification, and always confirm before discussing parcel details.
Conciseness: Provide answers that are clear and concise, avoiding overly technical terms unless absolutely necessary.
Empathy in Communication: If there are delays or issues, prioritize empathy in your responses. Acknowledge customer concerns and provide steps to resolve the issue where possible.
Accuracy: Ensure all tracking information, shipping costs, and delivery statuses are up-to-date and accurate.
Limit Jargon: Use clear language and avoid logistics jargon that could confuse customers, especially in complex scenarios.
Examples
Parcel Tracking Inquiry

Customer: "Where is my parcel?"
LogiLynk: "I’d be happy to track that for you! Could you please provide the tracking number? I’ll locate your package and give you the latest update, including an estimated delivery date."
Explanation of Delivery Status

Customer: "What does 'In Transit' mean?"
LogiLynk: "'In Transit' means your parcel is currently moving through our network and is on its way to the next location or directly to your delivery address. I’ll keep you posted with updates!"
Delivery Delay

Customer: "Why is my package delayed?"
LogiLynk: "I’m sorry for the delay! Delays can sometimes happen due to factors like high volume or weather conditions. Rest assured, your package is safe, and I’ll keep you updated as soon as it progresses. Thank you for your patience!"
Shipping Cost Calculation

Customer: "How much will it cost to send a 10kg package to California?"
LogiLynk: "I’d be happy to help! Could you also provide the package dimensions and let me know if you’d like standard or expedited delivery? I’ll calculate the cost once I have those details."
Issue Resolution for Lost Package

Customer: "I think my package is lost. What should I do?"
LogiLynk: "I understand how concerning that can be. I’ll assist in locating your package right away. Please provide your tracking number, and I’ll check our system for any updates. If necessary, I’ll initiate further steps to help resolve this."
"""

In [ ]:
struct = [{"role": "system", "content": System_Prompt}]

In [ ]:
dataframed = pd.read_csv('https://raw.githubusercontent.com/ALGOREX-PH/Day-4-AI-First-Dataset-Live/refs/heads/main/Parcel_Information_Dataset.csv')

In [ ]:
dataframed.head()

,Parcel ID,Sender Name,Sender Address,Sender Contact,Receiver Name,Receiver Address,Receiver Contact,Parcel Weight (kg),Parcel Dimensions (cm),Shipment Date,...,Shipping Method,Tracking Number,Insurance Value ($),Fragile,Temperature Sensitive,Special Instructions,Delivery Attempt Count,Last Location,Courier ID,combined
0,PCL398114,Michael Brown,"456 Maple Ave, Los Angeles, CA 90001",+1-555-4253,Sara Davis,"202 Birch Blvd, Phoenix, AZ 85001",+1-555-8367,7.75,97x16x70,2024-11-17,...,Standard,TRK389255020,257.34,Yes,No,NaN,1,"Distribution Center, Phoenix, AZ",CR9571,"PCL398114 Michael Brown 456 Maple Ave, Los Ang..."
1,PCL761243,John Doe,"101 Pine St, Houston, TX 77001",+1-555-2574,Sara Davis,"123 Elm St, New York, NY 10001",+1-555-3140,11.91,43x64x11,2024-11-23,...,Standard,TRK278841245,344.57,Yes,No,NaN,1,"Distribution Center, Phoenix, AZ",CR8429,"PCL761243 John Doe 101 Pine St, Houston, TX 77..."
2,PCL104639,John Doe,"202 Birch Blvd, Phoenix, AZ 85001",+1-555-3768,Bob Johnson,"123 Elm St, New York, NY 10001",+1-555-9256,5.65,45x39x53,2024-11-09,...,Express,TRK696430980,325.55,No,No,Deliver to reception,1,"Distribution Center, Phoenix, AZ",CR8897,"PCL104639 John Doe 202 Birch Blvd, Phoenix, AZ..."
3,PCL768961,Linda Black,"456 Maple Ave, Los Angeles, CA 90001",+1-555-2027,Sara Davis,"202 Birch Blvd, Phoenix, AZ 85001",+1-555-3973,15.05,23x99x83,2024-11-18,...,Express,TRK588614196,218.93,No,No,Handle with care,3,"Distribution Center, Houston, TX",CR4388,"PCL768961 Linda Black 456 Maple Ave, Los Angel..."
4,PCL935942,Alice Green,"101 Pine St, Houston, TX 77001",+1-555-3594,Mike Lee,"101 Pine St, Houston, TX 77001",+1-555-8828,13.20,43x83x93,2024-11-18,...,Express,TRK897242703,255.49,No,No,Deliver to reception,1,"Distribution Center, Phoenix, AZ",CR9727,"PCL935942 Alice Green 101 Pine St, Houston, TX..."


In [ ]:
dataframed['combined'] = dataframed.apply(lambda row : ' '.join(row.values.astype(str)), axis = 1)

In [ ]:
documents = dataframed['combined'].tolist()

In [ ]:
embeddings = [get_embedding(doc, engine = "text-embedding-3-small") for doc in documents]

In [ ]:
embedding_dim = len(embeddings[0])

In [ ]:
embeddings_np = np.array(embeddings).astype('float32')

In [ ]:
index = faiss.IndexFlatL2(embedding_dim)

In [ ]:
index.add(embeddings_np)

In [ ]:
user_message = "Hello my name is Michael Brown, Where is the parcel i sent Sara Davis?"

In [ ]:
query_embedding = get_embedding(user_message, engine='text-embedding-3-small')
query_embedding_np = np.array([query_embedding]).astype('float32')

In [ ]:
_, indices = index.search(query_embedding_np, 2)

In [ ]:
retrieved_docs = [documents[i] for i in indices[0]]

In [ ]:
context = ' '.join(retrieved_docs)

In [ ]:
structured_prompt = f"Context:\n{context}\n\nQuery:\n{user_message}\n\nResponse:"

In [ ]:
print(structured_prompt)

Context:
PCL264474 Michael Brown 789 Oak Rd, Chicago, IL 60601 +1-555-7874 Sara Davis 101 Pine St, Houston, TX 77001 +1-555-3769 3.56 92x75x48 2024-11-16 2024-11-12 Delivered 18.34 Express TRK638738696 265.1 Yes No Ring bell 2 Distribution Center, New York, NY CR9975 PCL398114 Michael Brown 456 Maple Ave, Los Angeles, CA 90001 +1-555-4253 Sara Davis 202 Birch Blvd, Phoenix, AZ 85001 +1-555-8367 7.75 97x16x70 2024-11-17 2024-11-17 Delivered 8.66 Standard TRK389255020 257.34 Yes No nan 1 Distribution Center, Phoenix, AZ CR9571

Query:
Hello my name is Michael Brown, Where is the parcel i sent Sara Davis?

Response:


In [ ]:
print(user_message)

Hello my name is Michael Brown, Where is the parcel i sent Sara Davis?


In [ ]:
chat =  openai.ChatCompletion.create(model = "gpt-4o-mini", messages = struct + [{"role": "user", "content" : structured_prompt}], temperature=0.5, max_tokens=1500, top_p=1, frequency_penalty=0, presence_penalty=0)
struct.append({"role": "user", "content": user_message})
response = chat.choices[0].message.content
struct.append({"role": "assistant", "content": response})

In [ ]:
print(response)

Hello, Michael! Thank you for your patience. I see that you sent two parcels to Sara Davis. Here are the details:

1. **Parcel with Tracking Number TRK638738696**: 
   - **Status**: Delivered
   - **Delivery Date**: November 12, 2024
   - **Destination**: 101 Pine St, Houston, TX 77001

2. **Parcel with Tracking Number TRK389255020**: 
   - **Status**: Delivered
   - **Delivery Date**: November 17, 2024
   - **Destination**: 202 Birch Blvd, Phoenix, AZ 85001

Both parcels have been successfully delivered. If you need any further assistance or have questions about these deliveries, feel free to ask!
